In [2]:
import sys
sys.path.append('../..')
import numpy as np
# PyTorch
import torch
import torch.nn.functional as F
from torch import cuda
from torch.autograd import Variable
from torchvision.models.mobilenet import mobilenet_v2

# DLBio and own scripts
from DLBio.pytorch_helpers import get_device
import transforms_data as td
from helpers import cuda_conv
import metrics as M
import acquire_ear_dataset as a

from PIL import Image
import glob


In [3]:
class Config():
    DEVICE = get_device()
    DATASET_DIR = '../dataset/'
    AUTH_DATASET_DIR = '../auth_dataset/unknown-auth/'
    MODEL_DIR = './models/ve_g_margin_2,0.pt'
    is_small_resize = False
    DATABASE_FOLDER = './embeddings/radius_2.0/'

In [4]:
model = torch.load(Config.MODEL_DIR, map_location=torch.device(Config.DEVICE))
transformation = td.get_transform('siamese_valid_and_test', Config.is_small_resize)

#model.eval()

In [5]:
def pipeline(input_, preprocess):
    input_ = input_.convert("L")
    input_ = preprocess(input_)
    input_ = input_.reshape(-1, td.get_resize(Config.is_small_resize)[0], td.get_resize(Config.is_small_resize)[1], 1)
    input_ = input_.permute(3, 0, 1, 2)

    if cuda.is_available():
        return input_.type('torch.cuda.FloatTensor')
    else:
        return input_.type('torch.FloatTensor')

In [11]:
# img = Image.open(Config.DATASET_DIR + 'janna_qua/janna_qua003.png')
# new_embedding = [model(Variable(pipeline(img,transformation))).cpu()]
# test = np.array(new_embedding)

# np.save('test.npy', test)

In [12]:
# loaded_test = np.load('test.npy', allow_pickle=True)

# print(loaded_test)
# print(new_embedding)

# value = F.pairwise_distance(loaded_test[0],new_embedding[0]).item()
# print(value)

In [10]:
result_value = []
result_label = []

img = Image.open(Config.AUTH_DATASET_DIR + 'unknown004.png')
new_embedding = model(Variable(pipeline(img,transformation))).cpu()

for label in os.listdir(Config.DATABASE_FOLDER):
    loaded_embedding = np.load(Config.DATABASE_FOLDER+label, allow_pickle=True)
    tmp = []    
    for embedding in loaded_embedding:
        dis = F.pairwise_distance(embedding,new_embedding)
        tmp.append(dis.item())
    result_value.append((min(tmp)))
    result_label.append(label)


In [13]:
result_value, result_label = zip(*sorted(zip(result_value, result_label)))
result_value = result_value[:10]
result_label = result_label[:10]

for idx, val in enumerate(result_label):
    print(str(idx+1) + ' : ' + ' ' + val + ' : ' + ' ' + str(result_value[idx]))

1 :  falco_len.npy :  0.6029162406921387
2 :  marina_han.npy :  0.6270309686660767
3 :  mohammed_muh.npy :  0.8420938849449158
4 :  moritz_mei.npy :  0.9366084337234497
5 :  linus_fal.npy :  1.0113420486450195
6 :  pauline_bus.npy :  1.0333360433578491
7 :  robert_sch.npy :  1.107620358467102
8 :  lars_fin.npy :  1.279685378074646
9 :  malte_gas.npy :  1.298358678817749
10 :  moritz_bor.npy :  1.344302773475647
